## Sberbank Russian Housing Market
#### Team Member: Zhian Wang; Ying Zhang
#### Data Source: https://www.kaggle.com/c/sberbank-russian-housing-market/data

In [1]:
import numpy as np
import pandas as pd
from sklearn import model_selection, preprocessing
import xgboost as xgb
import datetime
import pandas as pd

wd = "/Users/zwang/GWU/Kaggle/Housing_Mkt/data/"

### Model 01

In [3]:
#read csv files
train = pd.read_csv(wd+'train.csv')
test = pd.read_csv(wd+'test.csv')
id_test = test.id
id_train = train.id

mult = .969

y_train = train["price_doc"] * mult + 10
x_train = train.drop(["id", "timestamp", "price_doc"], axis=1)
x_test = test.drop(["id", "timestamp"], axis=1)

#Encode Categorical Variables
for c in x_train.columns:
    if x_train[c].dtype == 'object':
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(x_train[c].values))
        x_train[c] = lbl.transform(list(x_train[c].values))

for c in x_test.columns:
    if x_test[c].dtype == 'object':
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(x_test[c].values))
        x_test[c] = lbl.transform(list(x_test[c].values))

# Run XGB Model
xgb_params = {
    'eta': 0.05,
    'max_depth': 5,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'silent': 1
}

dtrain = xgb.DMatrix(x_train, y_train)
dtest = xgb.DMatrix(x_test)

cv_output = xgb.cv(xgb_params, dtrain, num_boost_round=1000, early_stopping_rounds=20, verbose_eval=25, show_stdv=False)
print('best num_boost_rounds = ', len(cv_output))
num_boost_rounds = len(cv_output) # 382

model01 = xgb.train(dict(xgb_params, silent=0), dtrain, num_boost_round= num_boost_rounds)

#y_predict = model01.predict(dtest)
#output01 = pd.DataFrame({'id': id_test, 'price_doc': y_predict01})

[0]	train-rmse:7.96311e+06	test-rmse:7.96868e+06
[25]	train-rmse:3.39599e+06	test-rmse:3.58856e+06
[50]	train-rmse:2.44957e+06	test-rmse:2.81627e+06
[75]	train-rmse:2.22139e+06	test-rmse:2.68314e+06
[100]	train-rmse:2.12427e+06	test-rmse:2.63849e+06
[125]	train-rmse:2.06274e+06	test-rmse:2.61565e+06
[150]	train-rmse:2.01144e+06	test-rmse:2.60251e+06
[175]	train-rmse:1.96638e+06	test-rmse:2.59128e+06
[200]	train-rmse:1.9241e+06	test-rmse:2.58258e+06
[225]	train-rmse:1.88682e+06	test-rmse:2.576e+06
[250]	train-rmse:1.85685e+06	test-rmse:2.57113e+06
[275]	train-rmse:1.82648e+06	test-rmse:2.5675e+06
[300]	train-rmse:1.79886e+06	test-rmse:2.56462e+06
[325]	train-rmse:1.77261e+06	test-rmse:2.56276e+06
[350]	train-rmse:1.74566e+06	test-rmse:2.56139e+06
[375]	train-rmse:1.72373e+06	test-rmse:2.56074e+06
[400]	train-rmse:1.70019e+06	test-rmse:2.56027e+06
best num_boost_rounds =  382


In [7]:
ytest_pred01 = model01.predict(dtrain)

df_train01 = pd.DataFrame({'id': id_train, 'price_pred01': ytrain_pred01})

df_train01.to_csv(wd+"/stacking/model01_train.csv", index=False)

ytest_pred01 = model01.predict(dtest)

df_test01 = pd.DataFrame({'id': id_test, 'price_pred01': ytest_pred01})

df_test01.to_csv(wd+"/stacking/model01_test.csv", index=False)

In [6]:
df_train01.head()

,id,price_pred01
0,1,5287640.0
1,2,5327946.5
2,3,5523237.5
3,4,10826237.0
4,5,13112374.0


### Model 02

In [13]:
#read  csv files
train = pd.read_csv(wd+'train.csv', parse_dates=['timestamp'])
test = pd.read_csv(wd+'test.csv', parse_dates=['timestamp'])
id_test = test.id

#preparing data
bad_index = train[train.life_sq > train.full_sq].index
train.loc[bad_index, "life_sq"] = np.NaN
equal_index = [601,1896,2791]
test.loc[equal_index, "life_sq"] = test.loc[equal_index, "full_sq"]
bad_index = test[test.life_sq > test.full_sq].index
test.loc[bad_index, "life_sq"] = np.NaN
bad_index = train[train.life_sq < 5].index
train.loc[bad_index, "life_sq"] = np.NaN
bad_index = test[test.life_sq < 5].index
test.loc[bad_index, "life_sq"] = np.NaN
bad_index = train[train.full_sq < 5].index
train.loc[bad_index, "full_sq"] = np.NaN
bad_index = test[test.full_sq < 5].index
test.loc[bad_index, "full_sq"] = np.NaN
kitch_is_build_year = [13117]
train.loc[kitch_is_build_year, "build_year"] = train.loc[kitch_is_build_year, "kitch_sq"]
bad_index = train[train.kitch_sq >= train.life_sq].index
train.loc[bad_index, "kitch_sq"] = np.NaN
bad_index = test[test.kitch_sq >= test.life_sq].index
test.loc[bad_index, "kitch_sq"] = np.NaN
bad_index = train[(train.kitch_sq == 0).values + (train.kitch_sq == 1).values].index
train.loc[bad_index, "kitch_sq"] = np.NaN
bad_index = test[(test.kitch_sq == 0).values + (test.kitch_sq == 1).values].index
test.loc[bad_index, "kitch_sq"] = np.NaN
bad_index = train[(train.full_sq > 210) & (train.life_sq / train.full_sq < 0.3)].index
train.loc[bad_index, "full_sq"] = np.NaN
bad_index = test[(test.full_sq > 150) & (test.life_sq / test.full_sq < 0.3)].index
test.loc[bad_index, "full_sq"] = np.NaN
bad_index = train[train.life_sq > 300].index
train.loc[bad_index, ["life_sq", "full_sq"]] = np.NaN
bad_index = test[test.life_sq > 200].index
test.loc[bad_index, ["life_sq", "full_sq"]] = np.NaN
train.product_type.value_counts(normalize= True)
test.product_type.value_counts(normalize= True)
bad_index = train[train.build_year < 1500].index
train.loc[bad_index, "build_year"] = np.NaN
bad_index = test[test.build_year < 1500].index
test.loc[bad_index, "build_year"] = np.NaN
bad_index = train[train.num_room == 0].index
train.loc[bad_index, "num_room"] = np.NaN
bad_index = test[test.num_room == 0].index
test.loc[bad_index, "num_room"] = np.NaN
bad_index = [10076, 11621, 17764, 19390, 24007, 26713, 29172]
train.loc[bad_index, "num_room"] = np.NaN
bad_index = [3174, 7313]
test.loc[bad_index, "num_room"] = np.NaN
bad_index = train[(train.floor == 0).values * (train.max_floor == 0).values].index
train.loc[bad_index, ["max_floor", "floor"]] = np.NaN
bad_index = train[train.floor == 0].index
train.loc[bad_index, "floor"] = np.NaN
bad_index = train[train.max_floor == 0].index
train.loc[bad_index, "max_floor"] = np.NaN
bad_index = test[test.max_floor == 0].index
test.loc[bad_index, "max_floor"] = np.NaN
bad_index = train[train.floor > train.max_floor].index
train.loc[bad_index, "max_floor"] = np.NaN
bad_index = test[test.floor > test.max_floor].index
test.loc[bad_index, "max_floor"] = np.NaN
train.floor.describe(percentiles= [0.9999])
bad_index = [23584]
train.loc[bad_index, "floor"] = np.NaN
train.material.value_counts()
test.material.value_counts()
train.state.value_counts()
bad_index = train[train.state == 33].index
train.loc[bad_index, "state"] = np.NaN
test.state.value_counts()

# removing extreme price per sqm
train.loc[train.full_sq == 0, 'full_sq'] = 50
train = train[train.price_doc/train.full_sq <= 600000]
train = train[train.price_doc/train.full_sq >= 10000]

# Add month-year
month_year = (train.timestamp.dt.month + train.timestamp.dt.year * 100)
month_year_cnt_map = month_year.value_counts().to_dict()
train['month_year_cnt'] = month_year.map(month_year_cnt_map)

month_year = (test.timestamp.dt.month + test.timestamp.dt.year * 100)
month_year_cnt_map = month_year.value_counts().to_dict()
test['month_year_cnt'] = month_year.map(month_year_cnt_map)

# Add week-year count
week_year = (train.timestamp.dt.weekofyear + train.timestamp.dt.year * 100)
week_year_cnt_map = week_year.value_counts().to_dict()
train['week_year_cnt'] = week_year.map(week_year_cnt_map)

week_year = (test.timestamp.dt.weekofyear + test.timestamp.dt.year * 100)
week_year_cnt_map = week_year.value_counts().to_dict()
test['week_year_cnt'] = week_year.map(week_year_cnt_map)

# Add month and day-of-week
train['month'] = train.timestamp.dt.month
train['dow'] = train.timestamp.dt.dayofweek

test['month'] = test.timestamp.dt.month
test['dow'] = test.timestamp.dt.dayofweek

# feature engineering - adding new fratures
train['rel_floor'] = train['floor'] / train['max_floor'].astype(float)
train['rel_kitch_sq'] = train['kitch_sq'] / train['full_sq'].astype(float)

test['rel_floor'] = test['floor'] / test['max_floor'].astype(float)
test['rel_kitch_sq'] = test['kitch_sq'] / test['full_sq'].astype(float)

train.apartment_name=train.sub_area + train['metro_km_avto'].astype(str)
test.apartment_name=test.sub_area + train['metro_km_avto'].astype(str)

train['room_size'] = train['life_sq'] / train['num_room'].astype(float)
test['room_size'] = test['life_sq'] / test['num_room'].astype(float)

train['year_old'] = 2018 - train['build_year']
test['year_old'] = 2018 - train['build_year']


rate_2016_q2 = 1
rate_2016_q1 = rate_2016_q2 / .99903
rate_2015_q4 = rate_2016_q1 / .9831
rate_2015_q3 = rate_2015_q4 / .9834
rate_2015_q2 = rate_2015_q3 / .9815
rate_2015_q1 = rate_2015_q2 / .9932
rate_2014_q4 = rate_2015_q1 / 1.0112
rate_2014_q3 = rate_2014_q4 / 1.0169
rate_2014_q2 = rate_2014_q3 / 1.0086
rate_2014_q1 = rate_2014_q2 / 1.0126
rate_2013_q4 = rate_2014_q1 / 0.9902
rate_2013_q3 = rate_2013_q4 / 1.0041
rate_2013_q2 = rate_2013_q3 / 1.0044
rate_2013_q1 = rate_2013_q2 / 1.0104
rate_2012_q4 = rate_2013_q1 / 0.9832
rate_2012_q3 = rate_2012_q4 / 1.0277
rate_2012_q2 = rate_2012_q3 / 1.0279
rate_2012_q1 = rate_2012_q2 / 1.0279
rate_2011_q4 = rate_2012_q1 / 1.076
rate_2011_q3 = rate_2011_q4 / 1.0236
rate_2011_q2 = rate_2011_q3 / 1
rate_2011_q1 = rate_2011_q2 / 1.011

# test data
test['average_q_price'] = 1

test_2016_q2_index = test.loc[test['timestamp'].dt.year == 2016].loc[test['timestamp'].dt.month >= 4].loc[test['timestamp'].dt.month <= 7].index
test.loc[test_2016_q2_index, 'average_q_price'] = rate_2016_q2

test_2016_q1_index = test.loc[test['timestamp'].dt.year == 2016].loc[test['timestamp'].dt.month >= 1].loc[test['timestamp'].dt.month < 4].index
test.loc[test_2016_q1_index, 'average_q_price'] = rate_2016_q1

test_2015_q4_index = test.loc[test['timestamp'].dt.year == 2015].loc[test['timestamp'].dt.month >= 10].loc[test['timestamp'].dt.month < 12].index
test.loc[test_2015_q4_index, 'average_q_price'] = rate_2015_q4

test_2015_q3_index = test.loc[test['timestamp'].dt.year == 2015].loc[test['timestamp'].dt.month >= 7].loc[test['timestamp'].dt.month < 10].index
test.loc[test_2015_q3_index, 'average_q_price'] = rate_2015_q3

test_2015_q2_index = test.loc[test['timestamp'].dt.year == 2015].loc[test['timestamp'].dt.month >= 4].loc[test['timestamp'].dt.month < 7].index
test.loc[test_2015_q2_index, 'average_q_price'] = rate_2015_q2

test_2015_q1_index = test.loc[test['timestamp'].dt.year == 2015].loc[test['timestamp'].dt.month >= 4].loc[test['timestamp'].dt.month < 7].index
test.loc[test_2015_q1_index, 'average_q_price'] = rate_2015_q1


# train 2015
train['average_q_price'] = 1

train_2015_q4_index = train.loc[train['timestamp'].dt.year == 2015].loc[train['timestamp'].dt.month >= 10].loc[train['timestamp'].dt.month <= 12].index
train.loc[train_2015_q4_index, 'average_q_price'] = rate_2015_q4

train_2015_q3_index = train.loc[train['timestamp'].dt.year == 2015].loc[train['timestamp'].dt.month >= 7].loc[train['timestamp'].dt.month < 10].index
train.loc[train_2015_q3_index, 'average_q_price'] = rate_2015_q3

train_2015_q2_index = train.loc[train['timestamp'].dt.year == 2015].loc[train['timestamp'].dt.month >= 4].loc[train['timestamp'].dt.month < 7].index
train.loc[train_2015_q2_index, 'average_q_price'] = rate_2015_q2

train_2015_q1_index = train.loc[train['timestamp'].dt.year == 2015].loc[train['timestamp'].dt.month >= 1].loc[train['timestamp'].dt.month < 4].index
train.loc[train_2015_q1_index, 'average_q_price'] = rate_2015_q1


# train 2014
train_2014_q4_index = train.loc[train['timestamp'].dt.year == 2014].loc[train['timestamp'].dt.month >= 10].loc[train['timestamp'].dt.month <= 12].index
train.loc[train_2014_q4_index, 'average_q_price'] = rate_2014_q4

train_2014_q3_index = train.loc[train['timestamp'].dt.year == 2014].loc[train['timestamp'].dt.month >= 7].loc[train['timestamp'].dt.month < 10].index
train.loc[train_2014_q3_index, 'average_q_price'] = rate_2014_q3

train_2014_q2_index = train.loc[train['timestamp'].dt.year == 2014].loc[train['timestamp'].dt.month >= 4].loc[train['timestamp'].dt.month < 7].index
train.loc[train_2014_q2_index, 'average_q_price'] = rate_2014_q2

train_2014_q1_index = train.loc[train['timestamp'].dt.year == 2014].loc[train['timestamp'].dt.month >= 1].loc[train['timestamp'].dt.month < 4].index
train.loc[train_2014_q1_index, 'average_q_price'] = rate_2014_q1


# train 2013
train_2013_q4_index = train.loc[train['timestamp'].dt.year == 2013].loc[train['timestamp'].dt.month >= 10].loc[train['timestamp'].dt.month <= 12].index
train.loc[train_2013_q4_index, 'average_q_price'] = rate_2013_q4

train_2013_q3_index = train.loc[train['timestamp'].dt.year == 2013].loc[train['timestamp'].dt.month >= 7].loc[train['timestamp'].dt.month < 10].index
train.loc[train_2013_q3_index, 'average_q_price'] = rate_2013_q3

train_2013_q2_index = train.loc[train['timestamp'].dt.year == 2013].loc[train['timestamp'].dt.month >= 4].loc[train['timestamp'].dt.month < 7].index
train.loc[train_2013_q2_index, 'average_q_price'] = rate_2013_q2

train_2013_q1_index = train.loc[train['timestamp'].dt.year == 2013].loc[train['timestamp'].dt.month >= 1].loc[train['timestamp'].dt.month < 4].index
train.loc[train_2013_q1_index, 'average_q_price'] = rate_2013_q1


# train 2012
train_2012_q4_index = train.loc[train['timestamp'].dt.year == 2012].loc[train['timestamp'].dt.month >= 10].loc[train['timestamp'].dt.month <= 12].index
train.loc[train_2012_q4_index, 'average_q_price'] = rate_2012_q4

train_2012_q3_index = train.loc[train['timestamp'].dt.year == 2012].loc[train['timestamp'].dt.month >= 7].loc[train['timestamp'].dt.month < 10].index
train.loc[train_2012_q3_index, 'average_q_price'] = rate_2012_q3

train_2012_q2_index = train.loc[train['timestamp'].dt.year == 2012].loc[train['timestamp'].dt.month >= 4].loc[train['timestamp'].dt.month < 7].index
train.loc[train_2012_q2_index, 'average_q_price'] = rate_2012_q2

train_2012_q1_index = train.loc[train['timestamp'].dt.year == 2012].loc[train['timestamp'].dt.month >= 1].loc[train['timestamp'].dt.month < 4].index
train.loc[train_2012_q1_index, 'average_q_price'] = rate_2012_q1


# train 2011
train_2011_q4_index = train.loc[train['timestamp'].dt.year == 2011].loc[train['timestamp'].dt.month >= 10].loc[train['timestamp'].dt.month <= 12].index
train.loc[train_2011_q4_index, 'average_q_price'] = rate_2011_q4

train_2011_q3_index = train.loc[train['timestamp'].dt.year == 2011].loc[train['timestamp'].dt.month >= 7].loc[train['timestamp'].dt.month < 10].index
train.loc[train_2011_q3_index, 'average_q_price'] = rate_2011_q3

train_2011_q2_index = train.loc[train['timestamp'].dt.year == 2011].loc[train['timestamp'].dt.month >= 4].loc[train['timestamp'].dt.month < 7].index
train.loc[train_2011_q2_index, 'average_q_price'] = rate_2011_q2

train_2011_q1_index = train.loc[train['timestamp'].dt.year == 2011].loc[train['timestamp'].dt.month >= 1].loc[train['timestamp'].dt.month < 4].index
train.loc[train_2011_q1_index, 'average_q_price'] = rate_2011_q1

train['price_doc'] = train['price_doc'] * train['average_q_price']

print('price changed done')

y_train = train["price_doc"]
id_train = train["id"]
x_train = train.drop(["id", "timestamp", "price_doc", "average_q_price"], axis=1)
x_test = test.drop(["id", "timestamp", "average_q_price"], axis=1)

num_train = len(x_train)
x_all = pd.concat([x_train, x_test])

for c in x_all.columns:
    if x_all[c].dtype == 'object':
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(x_all[c].values))
        x_all[c] = lbl.transform(list(x_all[c].values))

x_train = x_all[:num_train]
x_test = x_all[num_train:]


xgb_params = {
    'eta': 0.05,
    'max_depth': 6,
    'subsample': 0.6,
    'colsample_bytree': 1,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'silent': 1
}

dtrain = xgb.DMatrix(x_train, y_train)
dtest = xgb.DMatrix(x_test)

cv_output = xgb.cv(xgb_params, dtrain, num_boost_round=1000, early_stopping_rounds=20,
verbose_eval=20, show_stdv=False)
print('best num_boost_rounds = ', len(cv_output))
num_boost_rounds = len(cv_output) 

model02 = xgb.train(dict(xgb_params, silent=0), dtrain, num_boost_round=num_boost_rounds)

#y_predict = model2.predict(dtest)
#gunja_output = pd.DataFrame({'id': id_test, 'price_doc': y_predict})


price changed done
[0]	train-rmse:8.36817e+06	test-rmse:8.37654e+06
[20]	train-rmse:3.87109e+06	test-rmse:4.06809e+06
[40]	train-rmse:2.55887e+06	test-rmse:2.95699e+06
[60]	train-rmse:2.19696e+06	test-rmse:2.7154e+06
[80]	train-rmse:2.0698e+06	test-rmse:2.65022e+06
[100]	train-rmse:1.99866e+06	test-rmse:2.61855e+06
[120]	train-rmse:1.93928e+06	test-rmse:2.60026e+06
[140]	train-rmse:1.88787e+06	test-rmse:2.58727e+06
[160]	train-rmse:1.84485e+06	test-rmse:2.57806e+06
[180]	train-rmse:1.80239e+06	test-rmse:2.57017e+06
[200]	train-rmse:1.76422e+06	test-rmse:2.56489e+06
[220]	train-rmse:1.72663e+06	test-rmse:2.56031e+06
[240]	train-rmse:1.68808e+06	test-rmse:2.55718e+06
[260]	train-rmse:1.65692e+06	test-rmse:2.55373e+06
[280]	train-rmse:1.62677e+06	test-rmse:2.55215e+06
[300]	train-rmse:1.60138e+06	test-rmse:2.55103e+06
[320]	train-rmse:1.57315e+06	test-rmse:2.55012e+06
[340]	train-rmse:1.54578e+06	test-rmse:2.54815e+06
best num_boost_rounds =  341


In [14]:
ytrain_pred02 = model02.predict(dtrain)

df_train02 = pd.DataFrame({'id': id_train, 'price_pred02': ytrain_pred02})

df_train02.to_csv(wd+"/stacking/model02_train.csv", index=False)

ytest_pred02 = model02.predict(dtest)

df_test02 = pd.DataFrame({'id': id_test, 'price_pred02': ytest_pred02})

df_test02.to_csv(wd+"/stacking/model02_test.csv", index=False)

### Model 03

In [15]:
# read csv files
df_train = pd.read_csv(wd+"/train.csv", parse_dates=['timestamp'])
df_test = pd.read_csv(wd+"test.csv", parse_dates=['timestamp'])
df_macro = pd.read_csv(wd+"macro.csv", parse_dates=['timestamp'])

df_train.drop(df_train[df_train["life_sq"] > 7000].index, inplace=True)

mult = 0.969
y_train = df_train['price_doc'].values * mult + 10
id_test = df_test['id']
id_train = df_train['id']

df_train.drop(['id', 'price_doc'], axis=1, inplace=True)
df_test.drop(['id'], axis=1, inplace=True)

num_train = len(df_train)
df_all = pd.concat([df_train, df_test])
df_all = df_all.join(df_macro, on='timestamp', rsuffix='_macro')
print(df_all.shape)

# Add month-year
month_year = (df_all.timestamp.dt.month + df_all.timestamp.dt.year * 100)
month_year_cnt_map = month_year.value_counts().to_dict()
df_all['month_year_cnt'] = month_year.map(month_year_cnt_map)

# Add week-year count
week_year = (df_all.timestamp.dt.weekofyear + df_all.timestamp.dt.year * 100)
week_year_cnt_map = week_year.value_counts().to_dict()
df_all['week_year_cnt'] = week_year.map(week_year_cnt_map)

# Add month and day-of-week
df_all['month'] = df_all.timestamp.dt.month
df_all['dow'] = df_all.timestamp.dt.dayofweek

# Other feature engineering
df_all['rel_floor'] = df_all['floor'] / df_all['max_floor'].astype(float)
df_all['rel_kitch_sq'] = df_all['kitch_sq'] / df_all['full_sq'].astype(float)

train['building_name'] = pd.factorize(train.sub_area + train['metro_km_avto'].astype(str))[0]
test['building_name'] = pd.factorize(test.sub_area + test['metro_km_avto'].astype(str))[0]

def add_time_features(col):
   col_month_year = pd.Series(pd.factorize(train[col].astype(str) + month_year.astype(str))[0])
   train[col + '_month_year_cnt'] = col_month_year.map(col_month_year.value_counts())

   col_week_year = pd.Series(pd.factorize(train[col].astype(str) + week_year.astype(str))[0])
   train[col + '_week_year_cnt'] = col_week_year.map(col_week_year.value_counts())

add_time_features('building_name')
add_time_features('sub_area')

def add_time_features(col):
   col_month_year = pd.Series(pd.factorize(test[col].astype(str) + month_year.astype(str))[0])
   test[col + '_month_year_cnt'] = col_month_year.map(col_month_year.value_counts())

   col_week_year = pd.Series(pd.factorize(test[col].astype(str) + week_year.astype(str))[0])
   test[col + '_week_year_cnt'] = col_week_year.map(col_week_year.value_counts())

add_time_features('building_name')
add_time_features('sub_area')

# Drop timestamp column 
df_all.drop(['timestamp', 'timestamp_macro'], axis=1, inplace=True)


factorize = lambda t: pd.factorize(t[1])[0]

df_obj = df_all.select_dtypes(include=['object'])

X_all = np.c_[
    df_all.select_dtypes(exclude=['object']).values,
    np.array(list(map(factorize, df_obj.iteritems()))).T
]
print(X_all.shape)

X_train = X_all[:num_train]
X_test = X_all[num_train:]


# Deal with categorical values
df_numeric = df_all.select_dtypes(exclude=['object'])
df_obj = df_all.select_dtypes(include=['object']).copy()

for c in df_obj:
    df_obj[c] = pd.factorize(df_obj[c])[0]

df_values = pd.concat([df_numeric, df_obj], axis=1)


# Convert to numpy values
X_all = df_values.values
print(X_all.shape)

X_train = X_all[:num_train]
X_test = X_all[num_train:]

df_columns = df_values.columns

# Run XGB Model
xgb_params = {
    'eta': 0.05,
    'max_depth': 5,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'silent': 1
}

dtrain = xgb.DMatrix(X_train, y_train, feature_names=df_columns)
dtest = xgb.DMatrix(X_test, feature_names=df_columns)

cv_output = xgb.cv(xgb_params, dtrain, num_boost_round=1000, early_stopping_rounds=20, verbose_eval=25, show_stdv=False)
print('best num_boost_rounds = ', len(cv_output))
num_boost_rounds = len(cv_output) #

model03 = xgb.train(dict(xgb_params, silent=0), dtrain, num_boost_round=num_boost_rounds)

#y_pred = model3.predict(dtest)
#df_sub = pd.DataFrame({'id': id_test, 'price_doc': y_pred})

(38132, 390)
(38132, 394)
(38132, 394)
[0]	train-rmse:7.95097e+06	test-rmse:7.95387e+06
[25]	train-rmse:3.37924e+06	test-rmse:3.56281e+06
[50]	train-rmse:2.44345e+06	test-rmse:2.80129e+06
[75]	train-rmse:2.21997e+06	test-rmse:2.67461e+06
[100]	train-rmse:2.11538e+06	test-rmse:2.62903e+06
[125]	train-rmse:2.04485e+06	test-rmse:2.60543e+06
[150]	train-rmse:1.99219e+06	test-rmse:2.5905e+06
[175]	train-rmse:1.94894e+06	test-rmse:2.57984e+06
[200]	train-rmse:1.90718e+06	test-rmse:2.56996e+06
[225]	train-rmse:1.86744e+06	test-rmse:2.5646e+06
[250]	train-rmse:1.83353e+06	test-rmse:2.55781e+06
[275]	train-rmse:1.80126e+06	test-rmse:2.55367e+06
[300]	train-rmse:1.77318e+06	test-rmse:2.55144e+06
[325]	train-rmse:1.74344e+06	test-rmse:2.54833e+06
[350]	train-rmse:1.71692e+06	test-rmse:2.54686e+06
[375]	train-rmse:1.69138e+06	test-rmse:2.54489e+06
[400]	train-rmse:1.66638e+06	test-rmse:2.54342e+06
[425]	train-rmse:1.64369e+06	test-rmse:2.54194e+06
[450]	train-rmse:1.62069e+06	test-rmse:2.53915e+06

In [16]:
ytrain_pred03 = model03.predict(dtrain)

df_train03 = pd.DataFrame({'id': id_train, 'price_pred03': ytrain_pred03})

df_train03.to_csv(wd+"/stacking/model03_train.csv", index=False)

ytest_pred03 = model03.predict(dtest)

df_test03 = pd.DataFrame({'id': id_test, 'price_pred03': ytest_pred03})

df_test03.to_csv(wd+"/stacking/model03_test.csv", index=False)

### Weighted Averaging 3 models

In [21]:
first_result = df_test01.merge(df_test03, on="id")
first_result["price_com1"] = np.exp( .714*np.log(first_result.price_pred01) +
                                    .286*np.log(first_result.price_pred03) )  
result = first_result.merge(df_test02, on="id")

result["price_doc"] = np.exp( .78*np.log(result.price_com1) +
                              .22*np.log(result.price_pred02) )
result["price_doc"] =result["price_doc"] *0.9925                              
result.drop(["price_pred01","price_pred03","price_com1","price_pred02"],axis=1,inplace=True)
result.head()
result.to_csv(wd+'Final_Submission.csv', index=False)

### Stacking 

In [22]:
macro_cols = ['balance_trade',  'average_provision_of_build_contract', 
              'micex_rgbi_tr', 'micex_cbi_tr', 'deposits_rate', 'mortgage_rate',
              'income_per_cap', 'rent_price_4+room_bus', 
              'gdp_annual','deposits_rate', ]

train_df = pd.read_csv(wd+'/train.csv')
test_df = pd.read_csv(wd+'/test.csv')
macro_df = pd.read_csv(wd+'/macro.csv',usecols=['timestamp'] + macro_cols)

id_test = test_df.id
id_all = pd.concat([train_df, test_df])['id']

model01_train = pd.read_csv(wd+"/stacking/model01_train.csv")
model01_train.columns = ['id', 'model01']
model01_test = pd.read_csv(wd+"/stacking/model01_test.csv")
model01_test.columns = ['id', 'model01']

model02_train = pd.read_csv(wd+"/stacking/model02_train.csv")
model02_train.columns = ['id', 'model02']
model02_test = pd.read_csv(wd+"/stacking/model02_test.csv")
model02_test.columns = ['id', 'model02']

model03_train = pd.read_csv(wd+"/stacking/model03_train.csv")
model03_train.columns = ['id', 'model03']
model03_test = pd.read_csv(wd+"/stacking/model03_test.csv")
model03_test.columns = ['id', 'model03']

model02_train["model02"].ix[model02_train["model02"]<0] = -model02_train["model02"]

train_df = pd.merge(train_df, model01_train, on='id', how='left')
train_df = pd.merge(train_df, model02_train, on='id', how='left')
train_df = pd.merge(train_df, model03_train, on='id', how='left')

test_df = pd.merge(test_df, model01_test, on='id', how='left')
test_df = pd.merge(test_df, model02_test, on='id', how='left')
test_df = pd.merge(test_df, model03_test, on='id', how='left')


/anaconda/lib/python3.6/site-packages/pandas/core/indexing.py:141: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [23]:
# Impute the outlier of full_sq, life_sq, kitch_sq with NaN
train_df['full_sq'].ix[train_df['full_sq'] > 2000]= np.nan
train_df['life_sq'].ix[train_df['life_sq'] > 2000]= np.nan
train_df['kitch_sq'].ix[train_df['kitch_sq'] > 500]= np.nan

train_df['life_sq'].ix[train_df['life_sq'] >= train_df['full_sq']] = np.nan
test_df['life_sq'].ix[test_df['life_sq'] >= test_df['full_sq']] = np.nan

train_df['kitch_sq'].ix[train_df['kitch_sq'] >= train_df['full_sq']] = np.nan
test_df['kitch_sq'].ix[test_df['kitch_sq'] >= test_df['full_sq']] = np.nan

train_df['rel_life_sq'] = train_df['life_sq'] / train_df['full_sq'].astype(float)
test_df['rel_life_sq'] = test_df['life_sq'] / test_df['full_sq'].astype(float)

train_df['rel_floor'] = train_df['floor'] / train_df['max_floor'].astype(float)
test_df['rel_floor'] = test_df['floor'] / test_df['max_floor'].astype(float)
train_df['rel_floor'].ix[train_df['rel_floor'] == np.inf] = np.nan
test_df['rel_floor'].ix[test_df['rel_floor'] == np.inf] = np.nan

train_df['rel_kitch_sq'] = train_df['kitch_sq'] / train_df['full_sq'].astype(float)
test_df['rel_kitch_sq'] = test_df['kitch_sq'] / test_df['full_sq'].astype(float)


train_df["year_old"] = 2020 - train_df["build_year"]
test_df["year_old"] = 2020 - test_df["build_year"]

train_df["avg_sq"] = train_df["full_sq"]/train_df["num_room"]
test_df["avg_sq"] = test_df["full_sq"]/test_df["num_room"]

train_df.loc[train_df['state'] == 33, 'state'] = train_df['state'].mode().iloc[0]

train_df.loc[train_df['build_year'] == 20052009, 'build_year'] = 2007
train_df.loc[train_df['build_year'] <= 1691, 'build_year'] = np.nan
test_df.loc[test_df['build_year'] <= 1691, 'build_year'] = np.nan

# Merge macro columns
train_df = pd.merge(train_df, macro_df, on='timestamp', how='left')
test_df = pd.merge(test_df, macro_df, on='timestamp', how='left')
                   
print(train_df.shape)
print(test_df.shape)

/anaconda/lib/python3.6/site-packages/pandas/core/indexing.py:141: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


(30471, 309)
(7662, 308)


In [38]:
y_train = train_df["price_doc"]
x_train = train_df.drop(["id", "timestamp", "price_doc"], axis=1)
x_test = test_df.drop(["id", "timestamp"], axis=1)

# Encode categorical variables
for c in x_train.columns:
    if x_train[c].dtype == 'object':
        lbl = preprocessing.LabelEncoder()
        lbl.fit(x_train[c].values.tolist()+x_test[c].values.tolist()) 
        x_train[c] = lbl.transform(list(x_train[c].values))
        
for c in x_test.columns:
    if x_test[c].dtype == 'object':
        lbl = preprocessing.LabelEncoder()
        lbl.fit(x_train[c].values.tolist()+x_test[c].values.tolist())
        x_test[c] = lbl.transform(list(x_test[c].values))

xgb_params = {
    'eta': 0.05,
    'max_depth': 6,
    'subsample': 0.7,
    'colsample_bytree': 1,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'silent': 1
}

df_all = pd.concat([x_train, x_test])
dtrain = xgb.DMatrix(x_train, y_train)
dtest = xgb.DMatrix(x_test)
dall = xgb.DMatrix(df_all)

cv_output = xgb.cv(xgb_params, dtrain, num_boost_round=1000, early_stopping_rounds=50, # 20
    verbose_eval=20, show_stdv=False)
cv_output[['train-rmse-mean', 'test-rmse-mean']].plot()

num_boost_rounds = len(cv_output)
model = xgb.train(dict(xgb_params, silent=0), dtrain, num_boost_round= num_boost_rounds)

y_predict = model.predict(dtest)
output = pd.DataFrame({'id': id_test, 'price_doc': y_predict})

#output.to_csv(wd+'/stacking.csv', index=False)
output

[0]	train-rmse:8.1708e+06	test-rmse:8.173e+06
[20]	train-rmse:3.32572e+06	test-rmse:3.38638e+06
[40]	train-rmse:1.85856e+06	test-rmse:2.00995e+06
[60]	train-rmse:1.50199e+06	test-rmse:1.71944e+06
[80]	train-rmse:1.40988e+06	test-rmse:1.66652e+06
[100]	train-rmse:1.36816e+06	test-rmse:1.6549e+06
[120]	train-rmse:1.33983e+06	test-rmse:1.65157e+06
[140]	train-rmse:1.31375e+06	test-rmse:1.6511e+06
[160]	train-rmse:1.28698e+06	test-rmse:1.65e+06
[180]	train-rmse:1.25725e+06	test-rmse:1.65009e+06
[200]	train-rmse:1.22974e+06	test-rmse:1.649e+06
[220]	train-rmse:1.19982e+06	test-rmse:1.64837e+06
[240]	train-rmse:1.17476e+06	test-rmse:1.64828e+06
[260]	train-rmse:1.15156e+06	test-rmse:1.64867e+06


,id,price_doc
0,30474,5.152947e+06
1,30475,8.377446e+06
2,30476,5.110135e+06
3,30477,5.996344e+06
4,30478,4.649760e+06
5,30479,8.319586e+06
6,30480,4.110743e+06
7,30481,4.062334e+06
8,30482,4.513150e+06
9,30483,4.441862e+06


In [39]:
output['price_doc'] = output['price_doc'].abs()

In [40]:
output.to_csv(wd+'/stacking4.csv', index=False)

#### Stacking did not improve the score in LeardBoard, So we use the weighted averaging result.